# EDA Step 2: 데이터 품질 점검

**목적**: 모든 분석의 기반을 다지는 단계
- 결측치 비율과 패턴
- 이상치 탐지
- 중복 여부
- 타입 오류 / 값 범위 확인

**Step 1에서 발견한 의문점**:
- year_of_publication이 float → 결측치?
- age가 float → 결측치?

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 데이터 로드
books = pd.read_csv('../data/books.csv')
users = pd.read_csv('../data/users.csv')
train = pd.read_csv('../data/train_ratings.csv')
test = pd.read_csv('../data/test_ratings.csv')

---
## 2.1 결측치 분석

In [2]:
def missing_summary(df, name):
    """결측치 요약 함수"""
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    
    result = pd.DataFrame({
        'Missing Count': missing,
        'Missing %': missing_pct,
        'Non-Null Count': len(df) - missing,
        'Dtype': df.dtypes
    })
    
    print(f"\n{'='*60}")
    print(f"[{name}] 결측치 현황 (총 {len(df):,}행)")
    print(f"{'='*60}")
    print(result)
    print(f"\n→ 결측치가 있는 컬럼: {list(result[result['Missing Count'] > 0].index)}")
    return result

# 각 데이터셋 결측치 확인
books_missing = missing_summary(books, 'books.csv')
users_missing = missing_summary(users, 'users.csv')
train_missing = missing_summary(train, 'train_ratings.csv')
test_missing = missing_summary(test, 'test_ratings.csv')


[books.csv] 결측치 현황 (총 149,570행)
                     Missing Count  Missing %  Non-Null Count    Dtype
isbn                             0       0.00          149570   object
book_title                       0       0.00          149570   object
book_author                      1       0.00          149569   object
year_of_publication              0       0.00          149570  float64
publisher                        0       0.00          149570   object
img_url                          0       0.00          149570   object
language                     67227      44.95           82343   object
category                     68851      46.03           80719   object
summary                      67227      44.95           82343   object
img_path                         0       0.00          149570   object

→ 결측치가 있는 컬럼: ['book_author', 'language', 'category', 'summary']

[users.csv] 결측치 현황 (총 68,092행)
          Missing Count  Missing %  Non-Null Count    Dtype
user_id               0     

In [3]:
# 결측치 전체 요약
print("\n" + "="*60)
print("결측치 전체 요약")
print("="*60)

summary_data = []
for name, df in [('books', books), ('users', users), ('train', train), ('test', test)]:
    for col in df.columns:
        missing_cnt = df[col].isnull().sum()
        if missing_cnt > 0:
            summary_data.append({
                'Dataset': name,
                'Column': col,
                'Missing Count': missing_cnt,
                'Missing %': round(missing_cnt / len(df) * 100, 2)
            })

if summary_data:
    missing_df = pd.DataFrame(summary_data)
    display(missing_df.sort_values('Missing %', ascending=False))
else:
    print("결측치가 있는 컬럼이 없습니다.")


결측치 전체 요약


,Dataset,Column,Missing Count,Missing %
2,books,category,68851,46.03
3,books,summary,67227,44.95
1,books,language,67227,44.95
4,users,age,27833,40.88
0,books,book_author,1,0.00


---
## 2.2 이상치 탐지: books.csv

In [4]:
print("="*60)
print("[books.csv] year_of_publication 분석")
print("="*60)

year = books['year_of_publication']

print(f"\n기본 통계:")
print(year.describe())

print(f"\n결측치: {year.isnull().sum():,}개 ({year.isnull().sum()/len(year)*100:.2f}%)")

# 이상한 연도 확인
print(f"\n[이상치 후보]")
print(f"- 0 또는 음수: {(year <= 0).sum():,}개")
print(f"- 1900년 이전: {((year > 0) & (year < 1900)).sum():,}개")
print(f"- 2025년 이후: {(year > 2025).sum():,}개")

# 실제 이상 값 샘플
print(f"\n[이상 연도 샘플]")
abnormal_years = books[(year <= 0) | (year > 2025) | (year < 1800)]['year_of_publication'].value_counts().head(10)
print(abnormal_years)

[books.csv] year_of_publication 분석

기본 통계:
count    149570.000000
mean       1994.590606
std           8.179733
min        1376.000000
25%        1991.000000
50%        1996.000000
75%        2000.000000
max        2006.000000
Name: year_of_publication, dtype: float64

결측치: 0개 (0.00%)

[이상치 후보]
- 0 또는 음수: 0개
- 1900년 이전: 3개
- 2025년 이후: 0개

[이상 연도 샘플]
year_of_publication
1378.0    1
1376.0    1
Name: count, dtype: int64


In [5]:
# 연도 분포 구간별 확인
print("\n[연도 분포 구간별]")
bins = [0, 1900, 1950, 1980, 1990, 2000, 2005, 2010, 2020, 3000]
labels = ['~1900', '1900-1950', '1950-1980', '1980-1990', '1990-2000', '2000-2005', '2005-2010', '2010-2020', '2020~']

year_binned = pd.cut(year.dropna(), bins=bins, labels=labels, right=False)
print(year_binned.value_counts().sort_index())


[연도 분포 구간별]
year_of_publication
~1900            3
1900-1950      168
1950-1980     6931
1980-1990    23809
1990-2000    72358
2000-2005    46281
2005-2010       20
2010-2020        0
2020~            0
Name: count, dtype: int64


In [6]:
print("\n" + "="*60)
print("[books.csv] language 분석")
print("="*60)

print(f"\n고유값 수: {books['language'].nunique()}")
print(f"결측치: {books['language'].isnull().sum():,}개 ({books['language'].isnull().sum()/len(books)*100:.2f}%)")
print(f"\n언어 분포 (상위 15개):")
print(books['language'].value_counts().head(15))


[books.csv] language 분석

고유값 수: 26
결측치: 67,227개 (44.95%)

언어 분포 (상위 15개):
language
en       78823
de        1282
es        1017
fr         883
it         123
nl          67
pt          47
da          37
ca          23
ms          10
no           6
zh-CN        3
la           3
ru           3
ja           3
Name: count, dtype: int64


In [7]:
print("\n" + "="*60)
print("[books.csv] category 분석")
print("="*60)

print(f"\n고유값 수: {books['category'].nunique()}")
print(f"결측치: {books['category'].isnull().sum():,}개 ({books['category'].isnull().sum()/len(books)*100:.2f}%)")
print(f"\n카테고리 분포 (상위 20개):")
print(books['category'].value_counts().head(20))


[books.csv] category 분석

고유값 수: 4292
결측치: 68,851개 (46.03%)

카테고리 분포 (상위 20개):
category
['Fiction']                      32956
['Juvenile Fiction']              5804
['Biography & Autobiography']     3320
['History']                       1925
['Religion']                      1818
['Juvenile Nonfiction']           1417
['Social Science']                1231
['Humor']                         1161
['Body, Mind & Spirit']           1109
['Business & Economics']          1070
['Cooking']                       1025
['Health & Fitness']               968
['Family & Relationships']         959
['Computers']                      730
['Travel']                         651
['Self-Help']                      640
['Psychology']                     635
['Poetry']                         626
['Science']                        624
['Art']                            562
Name: count, dtype: int64


In [8]:
# category 형식 확인 (리스트 형태인지)
print("\n[category 형식 샘플]")
print(books['category'].dropna().head(10).tolist())

# 문자열로 저장된 리스트인지 확인
sample_cat = books['category'].dropna().iloc[0]
print(f"\n첫 번째 category 타입: {type(sample_cat)}")
print(f"값: {sample_cat}")


[category 형식 샘플]
["['Actresses']", "['1940-1949']", "['Medical']", "['Fiction']", "['History']", "['Fiction']", "['Fiction']", "['Nature']", "['Fiction']", "['Fiction']"]

첫 번째 category 타입: <class 'str'>
값: ['Actresses']


In [9]:
print("\n" + "="*60)
print("[books.csv] summary 분석")
print("="*60)

print(f"\n결측치: {books['summary'].isnull().sum():,}개 ({books['summary'].isnull().sum()/len(books)*100:.2f}%)")

# summary 길이 분포
summary_lengths = books['summary'].dropna().str.len()
print(f"\nsummary 길이 통계:")
print(summary_lengths.describe())

print(f"\n길이가 매우 짧은 summary (10자 미만): {(summary_lengths < 10).sum():,}개")
print(f"길이가 매우 긴 summary (1000자 이상): {(summary_lengths >= 1000).sum():,}개")


[books.csv] summary 분석

결측치: 67,227개 (44.95%)

summary 길이 통계:
count    82343.000000
mean       172.169013
std         59.660750
min          1.000000
25%        126.000000
50%        181.000000
75%        227.000000
max        374.000000
Name: summary, dtype: float64

길이가 매우 짧은 summary (10자 미만): 90개
길이가 매우 긴 summary (1000자 이상): 0개


---
## 2.3 이상치 탐지: users.csv

In [10]:
print("="*60)
print("[users.csv] age 분석")
print("="*60)

age = users['age']

print(f"\n기본 통계:")
print(age.describe())

print(f"\n결측치: {age.isnull().sum():,}개 ({age.isnull().sum()/len(age)*100:.2f}%)")

# 이상한 나이 확인
print(f"\n[이상치 후보]")
print(f"- 0 이하: {(age <= 0).sum():,}개")
print(f"- 5세 미만: {((age > 0) & (age < 5)).sum():,}개")
print(f"- 100세 이상: {(age >= 100).sum():,}개")
print(f"- 120세 이상: {(age >= 120).sum():,}개")

[users.csv] age 분석

기본 통계:
count    40259.000000
mean        36.069873
std         13.842571
min          5.000000
25%         25.000000
50%         34.000000
75%         45.000000
max         99.000000
Name: age, dtype: float64

결측치: 27,833개 (40.88%)

[이상치 후보]
- 0 이하: 0개
- 5세 미만: 0개
- 100세 이상: 0개
- 120세 이상: 0개


In [11]:
# 나이 분포 구간별
print("\n[나이 분포 구간별]")
age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 300]
age_labels = ['0-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100+']

age_binned = pd.cut(age.dropna(), bins=age_bins, labels=age_labels, right=False)
print(age_binned.value_counts().sort_index())


[나이 분포 구간별]
age
0-10         74
10-20      3976
20-30     10969
30-40     10824
40-50      6858
50-60      4960
60-70      2018
70-80       511
80-90        57
90-100       12
100+          0
Name: count, dtype: int64


In [12]:
# 극단적 나이 샘플 확인
print("\n[극단적 나이 샘플 (100세 이상)]")
display(users[users['age'] >= 100].head(10))

print("\n[극단적 나이 샘플 (5세 미만)]")
display(users[(users['age'] > 0) & (users['age'] < 5)].head(10))


[극단적 나이 샘플 (100세 이상)]


,user_id,location,age



[극단적 나이 샘플 (5세 미만)]


,user_id,location,age


In [13]:
print("\n" + "="*60)
print("[users.csv] location 분석")
print("="*60)

print(f"\n고유값 수: {users['location'].nunique():,}")
print(f"결측치: {users['location'].isnull().sum():,}개")

print(f"\nlocation 분포 (상위 20개):")
print(users['location'].value_counts().head(20))


[users.csv] location 분석

고유값 수: 18,368
결측치: 0개

location 분포 (상위 20개):
location
toronto, ontario, canada               616
portland, oregon, usa                  540
seattle, washington, usa               536
chicago, illinois, usa                 503
san francisco, california, usa         456
vancouver, british columbia, canada    428
san diego, california, usa             426
new york, new york, usa                416
london, england, united kingdom        412
houston, texas, usa                    387
ottawa, ontario, canada                381
calgary, alberta, canada               346
austin, texas, usa                     326
los angeles, california, usa           306
melbourne, victoria, australia         263
st. louis, missouri, usa               259
sydney, new south wales, australia     253
victoria, british columbia, canada     247
edmonton, alberta, canada              240
minneapolis, minnesota, usa            240
Name: count, dtype: int64


In [15]:
# location 파싱: city, state, country 분리
print("\n[location 파싱 분석]")

# 쉼표로 분리
location_split = users['location'].str.split(', ', expand=True)

print(f"분리된 컬럼 수: {len(location_split.columns)}")
print(f"컬럼 인덱스: {list(location_split.columns)}")

# 실제 분리된 컬럼 수 확인
print(f"\n[각 컬럼별 샘플]")
for i in range(len(location_split.columns)):
    print(f"컬럼 {i}: {location_split[i].dropna().head(3).tolist()}")

# 3개 이상 분리되는 케이스 확인
print(f"\n[3개 초과로 분리되는 location 샘플]")
over_3_parts = users[location_split[3].notna()] if 3 in location_split.columns else pd.DataFrame()
if len(over_3_parts) > 0:
    print(f"해당 케이스 수: {len(over_3_parts):,}개")
    print(over_3_parts['location'].head(10).tolist())
else:
    print("없음")

# 마지막 컬럼을 country로 가정하고 분석
last_col = location_split.columns[-1]
country_col = location_split[last_col]

print(f"\n[마지막 컬럼(추정 country) 분포 상위 15개]")
print(country_col.value_counts().head(15))

# n/a, 빈 값 등 확인
print(f"\n[country 이상값]")
print(f"- 'n/a' 포함: {country_col.str.lower().str.contains('n/a', na=False).sum():,}개")
print(f"- 빈 문자열: {(country_col == '').sum():,}개")
print(f"- 결측치: {country_col.isnull().sum():,}개")

# 2개 미만으로 분리되는 케이스 (city만 있는 경우 등)
print(f"\n[분리 개수별 분포]")
split_counts = location_split.notna().sum(axis=1)
print(split_counts.value_counts().sort_index())


[location 파싱 분석]
분리된 컬럼 수: 5
컬럼 인덱스: [0, 1, 2, 3, 4]

[각 컬럼별 샘플]
컬럼 0: ['timmins', 'ottawa', 'n/a']
컬럼 1: ['ontario', 'ontario', 'n/a']
컬럼 2: ['canada', 'canada', 'n/a']
컬럼 3: ['usa', 'united kingdom', 'united kingdom']
컬럼 4: ['usa', 'usa', 'canada']

[3개 초과로 분리되는 location 샘플]
해당 케이스 수: 192개
['twin falls, idaho, idaho, usa', 'coalville, leicestershire, england, united kingdom', 'tharston, norwich, england, united kingdom', 'paris, france, ile de france, france', 'chicago, il, illinois, usa', 'ramsey, huntingdon, cambridgeshire, united kingdom', 'silsden, keighley, england, united kingdom', 'aladinma, imo state, n/a, nigeria', 'fpo, ap, okinawa, japan', 'preston, lancashire, england, united kingdom']

[마지막 컬럼(추정 country) 분포 상위 15개]
4
usa       3
canada    2
spain     1
Name: count, dtype: int64

[country 이상값]
- 'n/a' 포함: 0개
- 빈 문자열: 0개
- 결측치: 68,086개

[분리 개수별 분포]
2     2093
3    65807
4      186
5        6
Name: count, dtype: int64


---
## 2.4 이상치 탐지: train_ratings.csv

In [16]:
print("="*60)
print("[train_ratings.csv] rating 분석")
print("="*60)

rating = train['rating']

print(f"\n기본 통계:")
print(rating.describe())

print(f"\n[rating 값 분포]")
print(rating.value_counts().sort_index())

print(f"\n[이상치 확인]")
print(f"- 1 미만: {(rating < 1).sum():,}개")
print(f"- 10 초과: {(rating > 10).sum():,}개")
print(f"- 범위 외 값: {((rating < 1) | (rating > 10)).sum():,}개")

[train_ratings.csv] rating 분석

기본 통계:
count    306795.000000
mean          7.069714
std           2.433217
min           1.000000
25%           6.000000
50%           8.000000
75%           9.000000
max          10.000000
Name: rating, dtype: float64

[rating 값 분포]
rating
1     13249
2     12929
3     10520
4     12707
5     14111
6     25311
7     52928
8     73593
9     48673
10    42774
Name: count, dtype: int64

[이상치 확인]
- 1 미만: 0개
- 10 초과: 0개
- 범위 외 값: 0개


In [17]:
# rating 분포 비율
print("\n[rating 분포 비율]")
rating_pct = (rating.value_counts().sort_index() / len(rating) * 100).round(2)
print(rating_pct)

print(f"\n평균 rating: {rating.mean():.2f}")
print(f"중앙값 rating: {rating.median():.0f}")


[rating 분포 비율]
rating
1      4.32
2      4.21
3      3.43
4      4.14
5      4.60
6      8.25
7     17.25
8     23.99
9     15.86
10    13.94
Name: count, dtype: float64

평균 rating: 7.07
중앙값 rating: 8


---
## 2.5 중복 데이터 확인

In [18]:
print("="*60)
print("중복 데이터 확인")
print("="*60)

# 완전 중복 행
print("\n[완전 중복 행]")
print(f"- books: {books.duplicated().sum():,}개")
print(f"- users: {users.duplicated().sum():,}개")
print(f"- train: {train.duplicated().sum():,}개")
print(f"- test: {test.duplicated().sum():,}개")

# Key 기준 중복
print("\n[Key 기준 중복]")
print(f"- books (isbn): {books['isbn'].duplicated().sum():,}개")
print(f"- users (user_id): {users['user_id'].duplicated().sum():,}개")
print(f"- train (user_id, isbn): {train.duplicated(subset=['user_id', 'isbn']).sum():,}개")
print(f"- test (user_id, isbn): {test.duplicated(subset=['user_id', 'isbn']).sum():,}개")

중복 데이터 확인

[완전 중복 행]
- books: 0개
- users: 0개
- train: 0개
- test: 0개

[Key 기준 중복]
- books (isbn): 0개
- users (user_id): 0개
- train (user_id, isbn): 0개
- test (user_id, isbn): 0개


In [19]:
# train과 test 간 중복 interaction 확인
print("\n[Train-Test 간 중복 interaction]")
train_pairs = set(zip(train['user_id'], train['isbn']))
test_pairs = set(zip(test['user_id'], test['isbn']))

overlap = train_pairs & test_pairs
print(f"- 중복 (user_id, isbn) 쌍: {len(overlap):,}개")

if len(overlap) > 0:
    print("\n⚠️ 경고: Train과 Test에 동일한 (user, item) 쌍이 존재합니다!")


[Train-Test 간 중복 interaction]
- 중복 (user_id, isbn) 쌍: 0개


---
## 2.6 메타데이터 커버리지 확인

In [20]:
print("="*60)
print("메타데이터 커버리지 (Train/Test의 user/item이 메타데이터에 있는지)")
print("="*60)

books_isbn = set(books['isbn'])
users_id = set(users['user_id'])

# Train 커버리지
train_isbn_in_books = train['isbn'].isin(books_isbn).sum()
train_user_in_users = train['user_id'].isin(users_id).sum()

print(f"\n[Train Set 커버리지]")
print(f"- isbn이 books에 있는 비율: {train_isbn_in_books:,} / {len(train):,} ({train_isbn_in_books/len(train)*100:.2f}%)")
print(f"- user_id가 users에 있는 비율: {train_user_in_users:,} / {len(train):,} ({train_user_in_users/len(train)*100:.2f}%)")

# Test 커버리지
test_isbn_in_books = test['isbn'].isin(books_isbn).sum()
test_user_in_users = test['user_id'].isin(users_id).sum()

print(f"\n[Test Set 커버리지]")
print(f"- isbn이 books에 있는 비율: {test_isbn_in_books:,} / {len(test):,} ({test_isbn_in_books/len(test)*100:.2f}%)")
print(f"- user_id가 users에 있는 비율: {test_user_in_users:,} / {len(test):,} ({test_user_in_users/len(test)*100:.2f}%)")

메타데이터 커버리지 (Train/Test의 user/item이 메타데이터에 있는지)

[Train Set 커버리지]
- isbn이 books에 있는 비율: 306,795 / 306,795 (100.00%)
- user_id가 users에 있는 비율: 306,795 / 306,795 (100.00%)

[Test Set 커버리지]
- isbn이 books에 있는 비율: 76,699 / 76,699 (100.00%)
- user_id가 users에 있는 비율: 76,699 / 76,699 (100.00%)


In [21]:
# 메타데이터가 없는 케이스 분석
print("\n[메타데이터 없는 케이스]")

train_no_book_meta = train[~train['isbn'].isin(books_isbn)]
train_no_user_meta = train[~train['user_id'].isin(users_id)]

print(f"\nTrain에서 books 메타데이터 없는 행: {len(train_no_book_meta):,}개")
if len(train_no_book_meta) > 0:
    print(f"  - 해당 isbn 수: {train_no_book_meta['isbn'].nunique():,}개")
    
print(f"\nTrain에서 users 메타데이터 없는 행: {len(train_no_user_meta):,}개")
if len(train_no_user_meta) > 0:
    print(f"  - 해당 user_id 수: {train_no_user_meta['user_id'].nunique():,}개")


[메타데이터 없는 케이스]

Train에서 books 메타데이터 없는 행: 0개

Train에서 users 메타데이터 없는 행: 0개


---
## 2.7 isbn 형식 검증

In [22]:
print("="*60)
print("ISBN 형식 분석 (ISBN vs ASIN 혼재 여부)")
print("="*60)

isbn_series = books['isbn']

# 길이 분포
isbn_lengths = isbn_series.str.len()
print(f"\n[ISBN 길이 분포]")
print(isbn_lengths.value_counts().sort_index())

# 숫자만으로 구성된 ISBN vs 알파벳 포함
numeric_only = isbn_series.str.match(r'^\d+$')
print(f"\n[ISBN 구성]")
print(f"- 숫자만: {numeric_only.sum():,}개 ({numeric_only.sum()/len(isbn_series)*100:.2f}%)")
print(f"- 알파벳 포함: {(~numeric_only).sum():,}개 ({(~numeric_only).sum()/len(isbn_series)*100:.2f}%)")

# 알파벳 포함 샘플
print(f"\n[알파벳 포함 ISBN 샘플]")
print(isbn_series[~numeric_only].head(10).tolist())

ISBN 형식 분석 (ISBN vs ASIN 혼재 여부)

[ISBN 길이 분포]
isbn
10    149570
Name: count, dtype: int64

[ISBN 구성]
- 숫자만: 137,632개 (92.02%)
- 알파벳 포함: 11,938개 (7.98%)

[알파벳 포함 ISBN 샘플]
['074322678X', '038078243X', '067176537X', '042518630X', '042511774X', '087113375X', '067976397X', '038572179X', '847223973X', '044651747X']


---
## Step 2 요약

**결과를 복사해서 공유해주세요!**

핵심 확인 사항:
1. 어떤 컬럼에 결측치가 많은가?
2. year_of_publication, age에 이상치가 있는가?
3. rating 분포가 정상적인가?
4. 중복 데이터가 있는가?
5. 메타데이터 커버리지는 충분한가?